<a href="https://colab.research.google.com/github/ebsenthil/langgraph-agent-class2/blob/main/s3-list-create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet boto3 langchain langgraph langchain_openai


In [ ]:
from google.colab import userdata

# Correct way to access values
AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION = userdata.get("AWS_DEFAULT_REGION")  # fallback default
openai_api_key = userdata.get('OPENAI_API_KEY')



In [ ]:
from langchain.tools import tool
import requests
from datetime import datetime, timedelta
import os

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_DEFAULT_REGION"] = AWS_DEFAULT_REGION


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo",api_key=openai_api_key)  # or "gpt-3.5-turbo"
response = llm.invoke("hi")
print(response.content)

In [ ]:
# ✅ Tool Functions for LangGraph

import boto3
from langchain.tools import tool

s3_client = boto3.client("s3")

@tool
def list_s3_buckets() -> str:
    """List all S3 buckets"""
    response = s3_client.list_buckets()
    buckets = [bucket["Name"] for bucket in response["Buckets"]]
    return f"S3 Buckets: {', '.join(buckets)}"

@tool
def create_s3_bucket(bucket_name: str) -> str:
    """Create an S3 bucket with the given name"""
    try:
        s3_client.create_bucket(Bucket=bucket_name)
        return f"✅ Bucket '{bucket_name}' created successfully!"
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [ ]:
tools = [list_s3_buckets,create_s3_bucket]
llm_with_tools=llm.bind_tools(tools)

In [ ]:
SYSTEM_PROMPT = """
You are an AI assistant that interacts with Amazon S3 using predefined tools. You can help users manage their S3 buckets by either listing existing buckets or creating a new one.

You have access to the following tools:

1. `list_s3_buckets`:
   - Use this tool to list all S3 buckets available in the user's AWS account.
   - Example inputs:
     - "Show me all my S3 buckets"
     - "List the buckets I have"
     - "What buckets are there in my AWS?"

2. `create_s3_bucket`:
   - Use this tool to create a new S3 bucket with the provided name.
   - You must extract the intended bucket name from the user input.
   - Bucket names must be globally unique and follow AWS naming conventions.
   - Example inputs:
     - "Create an S3 bucket named `my-test-bucket`"
     - "I want to make a bucket called `data-storage-demo`"
     - "Make a new bucket `langgraph-agent-test`"

Guidelines:
- If the user asks to create a bucket, ensure you extract the bucket name accurately.
- If the user asks to list, use `list_s3_buckets`.
- Always respond clearly and concisely with the result of the tool call.
- If the action is unclear, ask the user to clarify what they want to do with S3.

Only use the tools you have. Do not make up bucket contents, objects, or permissions.

"""

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState,StateGraph, END, START

In [ ]:
def function_1(state:MessagesState):

    user_question=state["messages"]

    input_question = [SYSTEM_PROMPT]+user_question

    response = llm_with_tools.invoke(input_question)

    return {
        "messages":[response]
    }

In [ ]:
builder=StateGraph(MessagesState)
builder.add_node("llm_decision_step",function_1)


In [ ]:
from langgraph.prebuilt import ToolNode
builder.add_node("tools",ToolNode(tools))

In [ ]:
builder.add_edge(START,"llm_decision_step")

In [ ]:
from langgraph.prebuilt import tools_condition
builder.add_conditional_edges(
    "llm_decision_step",
    tools_condition,
)

In [ ]:
builder.add_edge("tools","llm_decision_step")

In [ ]:
react_graph=builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(react_graph.get_graph().draw_mermaid_png()))

In [ ]:
message=[HumanMessage(content="can you list the s3 buckets?")]

In [ ]:
message=[HumanMessage(content="can you create an s3 with name langgraph-test-18jun?")]

In [ ]:
result=react_graph.invoke({"messages":message})

In [ ]:
result

In [ ]:
for m in result["messages"]:
    m.pretty_print()

In [ ]:
from IPython.display import Markdown, display

response = result["messages"][-1].content
display(Markdown(response))  # For Jupyter or Colab